# Quantum Chemistry with Paddle Quantum's qchem
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

qchem, which builds on top of Paddle Quantum, is designed to be a toolkit for quantum chemistry research in quantum computing era. It provides high level APIs for researchers who are interested in leveraging their current quantum chemsitry calculation with quantum computing power. And it also allows experts to write customized code. qchem is currently under active development, feel free to join us by opening issues or pull requests.

## Calculating ground state energy of a molecule
qchem provides various tools for you to do quantum chemistry calculation. Currently, qchem module supports the following molecular wave function ansatz:
* Hardware Efficient ansatz[<sup>1</sup>](#refer-1),
* Slater determinant ansatz[<sup>2</sup>](#refer-2),
* Unitary Coupled Cluster singles and doubles (UCCSD) ansatz[<sup>3</sup>](#refer-3).

Let's start from example and try to calculate the ground state energy of hydrogen molecule. First, we need to import qchem.

In [2]:
import paddle_quantum as pq
from paddle_quantum import qchem as pq_qchem
import warnings
warnings.filterwarnings("ignore")

Then, we need to provide chemical information required by running quantum chemistry calculation, including geometry, charge, basis function etc.

In [3]:
# define the geometry of hydrogen molecule, length unit use angstrom.
h2_geometry = "H 0.0 0.0 0.0; H 0.0 0.0 0.74"
basis_set = "sto-3g"
multiplicity = 1
charge = 0

Next, we need to choose an wavefunction ansatz for our hydrogen molecule. Let's choose `UCCSDModel` as the ansatz and use `MolEnergyLoss` as the loss function. `UCCSDModel` use Trotter-Suzuki method to build its wavefunction ansatz, see [here](https://qml.baidu.com/tutorials/quantum-simulation/hamiltonian-simulation-with-product-formula.html) if you want to know more about Trotter-Suzuki method.

In [ ]:
# Build UCCSD ansatz.
n_qubits = 4
n_electrons = 2
uccsd_ansatz = pq_qchem.UCCSDModel(n_qubits, n_electrons, n_trotter_steps=3)

# Setup the loss function
loss_fn = pq_qchem.MolEnergyLoss(h2_geometry, basis_set)

Finally, you can follow the optimization procedure [here](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html) and use any paddlepaddle optimizer to train the ansatz you have built.

In [5]:
# use paddlepaddle's optimizer
import paddle

optimizer = paddle.optimizer.Adam(parameters=uccsd_ansatz.parameters(), learning_rate=0.1)

# prepare the initial quantum state, e.g. |0000>
init_state = pq.state.computational_basis(n_qubits, 0)

# define the optimization steps
num_itr = 100
for itr in range(0, num_itr):
    # run quantum circuit to arrive at the final state
    state = uccsd_ansatz(init_state)
    # calculate loss
    loss = loss_fn(state)
    # backpropagate the gradient
    loss.backward()
    # update the ansatz's parameter
    optimizer.minimize(loss)
    # clear current gradient
    optimizer.clear_grad()
    print(f"The iter is {itr:3d}, loss is {loss.item():3.5f}.")
print("The theoretical value is -1.137283834485513.")

The iter is   0, loss is 0.71510.
The iter is   1, loss is 0.68522.
The iter is   2, loss is -0.33124.
The iter is   3, loss is -0.17871.
The iter is   4, loss is -0.46901.
The iter is   5, loss is -0.35388.
The iter is   6, loss is -0.49272.
The iter is   7, loss is -0.65490.
The iter is   8, loss is -0.74100.
The iter is   9, loss is -0.75261.
The iter is  10, loss is -0.91565.
The iter is  11, loss is -1.00675.
The iter is  12, loss is -0.86924.
The iter is  13, loss is -0.91493.
The iter is  14, loss is -1.00358.
The iter is  15, loss is -0.97359.
The iter is  16, loss is -0.99694.
The iter is  17, loss is -1.05768.
The iter is  18, loss is -1.07174.
The iter is  19, loss is -1.05568.
The iter is  20, loss is -1.05740.
The iter is  21, loss is -1.06293.
The iter is  22, loss is -1.06410.
The iter is  23, loss is -1.08027.
The iter is  24, loss is -1.07894.
The iter is  25, loss is -1.09911.
The iter is  26, loss is -1.09829.
The iter is  27, loss is -1.09884.
The iter is  28, loss 

You can now change the ansatz to `HardwareEfficientModel` and compare its energy with the one you just obtained.

## Calculating the ground state energy using Hartree Fock method
Hartree Fock method is often considered as the starting point for more accurate quantum chemistry calculations. In order to run Hartree Fock calculation in qchem, you just need to replace the ansatz and the loss function to `RHFSlaterDeterminantModel` and `RHFEnergyLoss` (**NOTE: You need PySCF be installed before running Hartree Fock calculation**, `pip install -U pyscf`).

In [6]:
# Build a Hartree Fock ansatz.
n_qubits = 4
n_electrons = 2
hartreefock_ansatz = pq_qchem.RHFSlaterDeterminantModel(n_qubits, n_electrons)

# Setup the loss function
loss_fn = pq_qchem.RHFEnergyLoss(h2_geometry, basis_set)

Overwritten attributes  multiplicity  of <class 'pyscf.gto.mole.Mole'>


In [7]:
# use paddlepaddle's optimizer
import paddle

optimizer = paddle.optimizer.Adam(parameters=hartreefock_ansatz.parameters(), learning_rate=0.1)

# prepare the initial quantum state, e.g. |1100>
init_state = pq.state.computational_basis(n_qubits, 12)

# define the optimization steps
num_itr = 100
for itr in range(0, num_itr):
    # run quantum circuit to arrive at the final state
    state = hartreefock_ansatz(init_state)
    # calculate loss
    loss = loss_fn(state)
    # backpropagate the gradient
    loss.backward()
    # update the ansatz's parameter
    optimizer.minimize(loss)
    # clear current gradient
    optimizer.clear_grad()
    print(f"The iter is {itr:3d}, loss is {loss.item():3.5f}.")
print("The theoretical value is -1.11675.")

The iter is   0, loss is 0.34341.
The iter is   1, loss is 0.26274.
The iter is   2, loss is 0.15970.
The iter is   3, loss is 0.03472.
The iter is   4, loss is -0.11024.
The iter is   5, loss is -0.27116.
The iter is   6, loss is -0.44170.
The iter is   7, loss is -0.61316.
The iter is   8, loss is -0.77486.
The iter is   9, loss is -0.91509.
The iter is  10, loss is -1.02291.
The iter is  11, loss is -1.09057.
The iter is  12, loss is -1.11626.
The iter is  13, loss is -1.10573.
The iter is  14, loss is -1.07116.
The iter is  15, loss is -1.02724.
The iter is  16, loss is -0.98662.
The iter is  17, loss is -0.95737.
The iter is  18, loss is -0.94303.
The iter is  19, loss is -0.94371.
The iter is  20, loss is -0.95742.
The iter is  21, loss is -0.98086.
The iter is  22, loss is -1.01012.
The iter is  23, loss is -1.04104.
The iter is  24, loss is -1.06973.
The iter is  25, loss is -1.09296.
The iter is  26, loss is -1.10862.
The iter is  27, loss is -1.11600.
The iter is  28, loss is

---
## References

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)

[3] Abhinav, Aspuru-Guzik, et al. "A Quantum Computing View on Unitary Coupled Cluster Theory" (https://arxiv.org/abs/2109.15176)